# Preprocessing, GridSearch and Training SpiderNet-6 on Private Data without B/W-tests features

In [1]:
import numpy as np
from numpy import array
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import math

import matplotlib.pylab as plt
from matplotlib.pyplot import cm
import seaborn as sns
import itertools
import collections
from collections import OrderedDict, Counter

import scipy.stats as stats
from scipy.stats import normaltest, kstest, ttest_ind, ttest_rel, mannwhitneyu, wilcoxon, levene, bartlett
from scipy.stats import chi2_contingency, fisher_exact, mode, pearsonr, f_oneway, kruskal, spearmanr

from rfpimp import *

from datetime import datetime
import re
from seaborn import heatmap
import random
import statsmodels.distributions.empirical_distribution as edf
from scipy.interpolate import interp1d
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.utils import resample
from functools import reduce
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score

from sklearn import metrics, preprocessing
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, log_loss, recall_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, PowerTransformer

from sklearn.utils import check_consistent_length, column_or_1d, assert_all_finite
from sklearn.utils.extmath import stable_cumsum
from sklearn.utils.multiclass import type_of_target
from sklearn.linear_model import LogisticRegression
import re
import gc

import os

#from sklearn.externals import joblib
import joblib

from importlib import reload
import Filter_and_Grid_Search
Filter_and_Grid_Search = reload(Filter_and_Grid_Search)
from Filter_and_Grid_Search import stratified_split
from Filter_and_Grid_Search import attributes_list, attributes_list_new
from Filter_and_Grid_Search import get_s_stat, get_PSI_stat, get_stats_by_month, get_stats, stable_unstable
from Filter_and_Grid_Search import stable_unstable_by_month_divide, union_datas, individual_hists_all 
from Filter_and_Grid_Search import paired_time_hists_by_month, statistics_with_target
from Filter_and_Grid_Search import receive_correlations, find_doubles_corr
from Filter_and_Grid_Search import statistics_with_target, attributes_list, attributes_list_new, make_standard
from Filter_and_Grid_Search import data_preprocessing_train, data_preprocessing_test
from Filter_and_Grid_Search import receive_correlations, find_doubles_corr
from Filter_and_Grid_Search import stratified_split, two_forests, turn_variables_with_values
from Filter_and_Grid_Search import find_meta_params, calculate_vif#, find_meta_params_mem
from Filter_and_Grid_Search import plot_meta_2d, data_preprocessing, find_ouliers_iqr
from Filter_and_Grid_Search import train_model_receive_stats, simple_b_score_risk
from Filter_and_Grid_Search import max_prof_corve, by_month_gini, check_attribute_list_cases

from Filter_and_Grid_Search import to_zip, br_correction, br_stat

from statsmodels.stats.proportion import proportions_ztest

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import pathlib
import psutil

C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
PATH_DATA = 'D:/Share/01 -Risk Desc Science/AntiFraud/Chinese_data/Datasets/'
PATH = 'D:/Share/safanasev/Python-notebook/AF_ML_chinese/'
print(PATH_DATA)
print(PATH)

D:/Share/01 -Risk Desc Science/AntiFraud/Chinese_data/Datasets/
D:/Share/safanasev/Python-notebook/AF_ML_chinese/


In [6]:
task = 'binary'
missing_strings = 'MISSING'
p_value = 0.05
target_dict = {'good': 0, 'bad': 1}

In [7]:
target = 'BAD_FLAG'
index_month = 'MONTH_YEAR'
list_of_vars_for_strat = ['MONTH_YEAR']
sort_by_var = 'APPPOSID'

necessary_fields = [target, index_month, sort_by_var]

COL_DEL = ['Unnamed: 0', 'PERIOD_7', 'LOSS_90P'] 
COL_DEL = [x.upper() for x in COL_DEL]
COL_TRG = target

In [8]:
COL_DEL

['UNNAMED: 0', 'PERIOD_7', 'LOSS_90P']

In [9]:
necessary_fields

['BAD_FLAG', 'MONTH_YEAR', 'APPPOSID']

In [10]:
n_jobs = 3

## Start of selection

In [11]:
from tensorflow.keras.layers import (Input, Activation, Dense, Flatten, Dropout)
from tensorflow.keras.layers import (Conv1D, MaxPooling1D, AveragePooling1D)
from tensorflow.keras.layers import add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K

In [12]:
from tensorflow.keras.layers import ReLU, Concatenate, GlobalAveragePooling1D

In [13]:
from keras.utils import np_utils
from tensorflow.keras.models import Model

In [14]:
from tensorflow.keras.callbacks import EarlyStopping, Callback, LearningRateScheduler
#from keras.callbacks import EarlyStopping, Callback

# объявляем класс метрик

class E_time(Callback):
    def on_train_begin(self, logs={}):
        tm = datetime.strftime(datetime.now(), "%d.%m.%Y %H:%M:%S")
        print ('train_begin', '| time: ' , tm)
        
    def on_epoch_end(self, epoch, logs={}):
        tm = datetime.strftime(datetime.now(), "%d.%m.%Y %H:%M:%S")
        print ('epoch_end', '| time: ' , tm)
    
        return

_time = E_time()

In [15]:
import tensorflow as tf

In [16]:
print(tf.__version__)

2.3.0


In [17]:
#если alpha не задано, то не делаем br_correction

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self,  dataset, x_col, y_col,  batch_size=256, num_classes=2, alpha = 0.1, random_state = 42, 
                 shuffle=True, class_w = None):
        self.batch_size = batch_size
        self.dataset = dataset
        self.x_col = x_col
        self.y_col = y_col
        self.indices = self.dataset.index.tolist()
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.alpha = alpha
        self.random_state = random_state
        self.class_w = class_w
        self.on_epoch_end()

    def __len__(self):
        return math.ceil(len(self.indices) / self.batch_size)

    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in index]
        
        data_tmp = self.dataset.loc[batch]
        data_tmp_b = data_tmp[data_tmp[self.y_col] == 1]
        data_tmp_b_all = self.dataset[self.dataset[self.y_col] == 1]
        X_tmp = data_tmp.head(0)
        
        
        #print(data_tmp_b.shape[0], data_tmp.shape[0], data_tmp_b_all.shape[0])
        if (self.alpha is None) and (self.class_w is None):
            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col] 
            
        elif (self.alpha is None) and self.class_w > 0:
            k = (self.class_w*self.batch_size)/(1-self.class_w)
            k = k - data_tmp_b.shape[0]
            if int(k) < 1:
                k = k+1
            ix2 = np.random.RandomState(self.random_state + int(k)).choice(data_tmp_b_all.shape[0], int(k))
            X_tmp = data_tmp_b_all.iloc[ix2]
            data_tmp = data_tmp.append(X_tmp, ignore_index=True)
            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col]
            #print(data_tmp_b.shape[0], data_tmp.shape[0],  data_tmp_b.shape[0]/(data_tmp.shape[0] - X_tmp.shape[0]), np.mean(y)) 
            
            
            
        else:
            if data_tmp_b.shape[0] > 0:
                k=(self.alpha*self.batch_size/data_tmp_b.shape[0] -1 )/(1-self.alpha)
                for i in range(0,int(k)):
                    X_tmp = X_tmp.append(data_tmp_b, ignore_index=True)
            else:
                k = (self.alpha*self.batch_size)/(1-self.alpha)
                ix2 = np.random.RandomState(self.random_state + int(k)).choice(data_tmp_b_all.shape[0], int(k))
                X_tmp = data_tmp_b_all.iloc[ix2]
                data_tmp_b = X_tmp

    #         print('Добавим дробное число строк')    
            k_fraction = k - int(k)
            n_samples = int(round(data_tmp_b.shape[0]*k_fraction))
    #         print(k_fraction, n_samples)

            ix = np.random.RandomState(self.random_state).choice(data_tmp_b.shape[0], n_samples)
            data_add_fraction = data_tmp_b.iloc[ix]
            X_tmp.append(data_add_fraction, ignore_index=True)

            data_tmp = data_tmp.append(X_tmp, ignore_index=True)


            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col]   
        #print(sum(self.dataset.loc[batch][self.y_col])/len(index), np.mean(y))
        
        if self.num_classes > 1:
            y = np_utils.to_categorical( y, self.num_classes)
            
        ####for tf 2.3.0, [np.array(X)], np.array(y)

        #return [np.array(X).reshape(X.shape[0], X.shape[1], 1)], np.array(y)
        return [np.array(X)], np.array(y)

    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)

In [18]:
from tensorflow.keras.callbacks import EarlyStopping

## Loading Data

In [19]:
PATH = 'D:/Share/safanasev/Python-notebook/AF_ML_v2_2014/'
#загружаем данные для 163 переменных

train_for = pd.read_csv(PATH + 'train_163_prep.csv')
valid_for = pd.read_csv(PATH + 'valid_163_prep.csv')
test_for = pd.read_csv(PATH + 'test_163_prep.csv')

In [20]:
y_train = train_for[target]
#y_test = valid_for[target]
#y_val = test_for[target]

y_test = test_for[target]
y_val = valid_for[target]

train_for.drop('Unnamed: 0', axis = 1, inplace = True)
valid_for.drop('Unnamed: 0', axis = 1, inplace = True)
test_for.drop('Unnamed: 0', axis = 1, inplace = True)


col = train_for.columns.to_list()
col.remove(target)


X_1_2 = train_for[col]
X_2_2 = valid_for[col]
X_3_2 = test_for[col]

In [21]:
#проверка корректности
X_1_2.shape, train_for.shape, len(y_train)

((1507599, 163), (1507599, 164), 1507599)

In [22]:
x_col = X_2_2.columns
y_col = 'BAD_FLAG'

In [23]:
#проверка корректности, должно быть везде True
print(X_2_2.shape[0] == valid_for.shape[0])
print(X_2_2.shape[1] == valid_for.shape[1] - 1)
print(len(y_val) == valid_for.shape[0])

True
True
True


In [24]:
#если alpha не задано, то не делаем br_correction
#новая версия, исправлены индексы

class DataGenerator_new(tf.keras.utils.Sequence):
    def __init__(self,  dataset, x_col, y_col,  batch_size=256, num_classes=2, alpha = 0.1, random_state = 42, 
                 shuffle=True, class_w = None):
        self.batch_size = batch_size
        self.dataset = dataset
        self.x_col = x_col
        self.y_col = y_col
        self.indices = self.dataset.index.tolist()
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.alpha = alpha
        self.random_state = random_state
        self.class_w = class_w
        self.on_epoch_end()

    def __len__(self):
        return math.ceil(len(self.indices) / self.batch_size)

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in indexes]
        
        data_tmp = self.dataset.loc[batch]
        data_tmp_b = data_tmp[data_tmp[self.y_col] == 1]
        data_tmp_b_all = self.dataset[self.dataset[self.y_col] == 1]
        X_tmp = data_tmp.head(0)
        
        
        #print(data_tmp_b.shape[0], data_tmp.shape[0], data_tmp_b_all.shape[0])
        if (self.alpha is None) and (self.class_w is None):
            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col] 
            
        elif (self.alpha is None) and self.class_w > 0:
            k = (self.class_w*self.batch_size)/(1-self.class_w)
            k = k - data_tmp_b.shape[0]
            if int(k) < 1:
                k = k+1
            ix2 = np.random.RandomState(self.random_state + int(k)).choice(data_tmp_b_all.shape[0], int(k))
            X_tmp = data_tmp_b_all.iloc[ix2]
            data_tmp = data_tmp.append(X_tmp, ignore_index=True)
            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col]
            #print(data_tmp_b.shape[0], data_tmp.shape[0],  data_tmp_b.shape[0]/(data_tmp.shape[0] - X_tmp.shape[0]), np.mean(y)) 
            
            
            
        else:
            if data_tmp_b.shape[0] > 0:
                k=(self.alpha*self.batch_size/data_tmp_b.shape[0] -1 )/(1-self.alpha)
                for i in range(0,int(k)):
                    X_tmp = X_tmp.append(data_tmp_b, ignore_index=True)
            else:
                k = (self.alpha*self.batch_size)/(1-self.alpha)
                ix2 = np.random.RandomState(self.random_state + int(k)).choice(data_tmp_b_all.shape[0], int(k))
                X_tmp = data_tmp_b_all.iloc[ix2]
                data_tmp_b = X_tmp

    #         print('Добавим дробное число строк')    
            k_fraction = k - int(k)
            n_samples = int(round(data_tmp_b.shape[0]*k_fraction))
    #         print(k_fraction, n_samples)

            ix = np.random.RandomState(self.random_state).choice(data_tmp_b.shape[0], n_samples)
            data_add_fraction = data_tmp_b.iloc[ix]
            X_tmp.append(data_add_fraction, ignore_index=True)

            data_tmp = data_tmp.append(X_tmp, ignore_index=True)


            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col]   
        #print(sum(self.dataset.loc[batch][self.y_col])/len(index), np.mean(y))
        
        if self.num_classes > 1:
            y = np_utils.to_categorical( y, self.num_classes)
            
        ####for tf 2.3.0, [np.array(X)], np.array(y)

        return [np.array(X).reshape(X.shape[0], X.shape[1], 1)], np.array(y)

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

In [25]:
training_generator = DataGenerator(train_for, x_col, y_col, batch_size=512, alpha = None, class_w = 0.05 )
#validation_generator = DataGenerator(valid_for, x_col, y_col, batch_size=valid_for.shape[0], alpha = None, class_w = None)

In [26]:
Y_test_2 = np_utils.to_categorical( y_val, 2) # преобразовываем в 2 класса 

In [27]:
X_2_2.shape, Y_test_2.shape

((185400, 163), (185400, 2))

In [28]:
b_test = []
w_test = []
del_all = []

for i in X_2_2.columns:
    if i.find('W_TEST') >= 0:
        w_test.append(i)
        del_all.append(i)
    if i.find('B_TEST') >= 0:
        b_test.append(i)
        del_all.append(i)

In [29]:
col = train_for.columns.to_list()
col.remove(target)

col_2 = [i for i in col if i not in del_all]


X_1_2 = train_for[col_2]
X_2_2 = valid_for[col_2]
X_3_2 = test_for[col_2]

In [30]:
X_1_2.shape, 163 - len(b_test) - len(w_test)

((1507599, 124), 124)

In [55]:
x_col = X_2_2.columns
y_col = 'BAD_FLAG'

In [56]:
len(x_col)

124

# *

In [31]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 29

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.compat.v1.set_random_seed(seed_value)


# 5. Configure a new global `tensorflow` session
# for later versions:
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

# ***

In [32]:
from tensorflow.keras.layers import GlobalMaxPooling1D

In [34]:
class Metrics(tf.keras.callbacks.Callback):
    def __init__(self, valid_data):
        super(Metrics, self).__init__()
        self.validation_data = valid_data

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        val_predict = self.model.predict(self.validation_data[0])[:, 1]
        val_targ = self.validation_data[1]
        
        if len(val_targ.shape) == 2: #and val_targ.shape[1] != 1:
            val_targ = val_targ[:,1]

        _val_aps = metrics.average_precision_score(val_targ, val_predict)
        #_val_recall = metrics.recall_score(val_targ, val_predict)
        _val_a = metrics.roc_auc_score(val_targ, val_predict)

        logs['val_aps'] = _val_aps
        logs['val_a'] = _val_a
        print(" — val_aps:  %f — val_a: %f" % (_val_aps, _val_a))
        return

In [35]:
import os
os.environ["PATH"] += os.pathsep + 'C:\\Program Files (x86)\\Graphviz2.38\\bin\\'

In [36]:
#training_aug = DataGenerator(aug_for, x_col, y_col, batch_size=512, alpha = None, class_w = 0.05 )
training_generator = DataGenerator(train_for, x_col, y_col, batch_size=512, alpha = None, class_w = 0.05 )

In [37]:
#learning_rate=0.005
initial_learning_rate = 0.005

def lr_step_decay(epoch, lr):
    drop_rate = 0.1
    epochs_drop = 20.0
    
    return initial_learning_rate * math.pow(drop_rate, math.floor(epoch/epochs_drop))

In [38]:
import urllib
import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState

In [39]:
from tensorflow.keras.models import load_model

## Grid Search

In [37]:
gl_p = [False] #0
l2_batch = [ 0.0002] #1
n_ker = [5, 7] #2
n_fil = [ 5, 15] #3
d_hidden = [30, 100] #4
drop_out = [0.30] #5
drop_out_conv = [0.001] #6
reg_dense = [ 0.0002] #7

In [38]:
param = itertools.product(gl_p, l2_batch, n_ker, n_fil, d_hidden, drop_out, drop_out_conv, reg_dense)

In [39]:
for i in param:
    print(i)

(False, 0.0002, 5, 5, 30, 0.3, 0.001, 0.0002)
(False, 0.0002, 5, 5, 100, 0.3, 0.001, 0.0002)
(False, 0.0002, 5, 15, 30, 0.3, 0.001, 0.0002)
(False, 0.0002, 5, 15, 100, 0.3, 0.001, 0.0002)
(False, 0.0002, 7, 5, 30, 0.3, 0.001, 0.0002)
(False, 0.0002, 7, 5, 100, 0.3, 0.001, 0.0002)
(False, 0.0002, 7, 15, 30, 0.3, 0.001, 0.0002)
(False, 0.0002, 7, 15, 100, 0.3, 0.001, 0.0002)


In [40]:
training_generator = DataGenerator(train_for, x_col, y_col, batch_size=512, alpha = None, class_w = 0.05 ) #было 0,05
#validation_generator = DataGenerator(valid_for, x_col, y_col, batch_size=valid_for.shape[0], alpha = None, class_w = None)

In [40]:
inp_shape = X_2_2.shape[1] #китайский датасэт
inp_shape

124

In [53]:
len(x_col)

163

## The best SpiderNet-6

In [41]:
def spider_model(l2_conv = None, reg = 1E-5 ,
                 _bias = True,  n_features = 163, n_pool = 2, n_kernel = 5, n_filters = 5, n_strides = 1,
                 classes = 2, hidden = 64 , drop_out = 0.25, drop_out_conv = 0.001, drop_out_rate = 4 ,
                 padding_pool = 'valid' ,
                gl_pool_max = False):
    
    if reg == None:
        l2_batch_gamma = None 
        l2_batch_betta = None
    else:
        l2_batch_gamma = l2(reg)
        l2_batch_betta =l2(reg)
    
    
    x = Input(shape=(  n_features, 1))
    n = 0
    y = Conv1D(filters=n_filters, kernel_size=n_kernel, strides=n_strides, padding='same', 
           use_bias=_bias, kernel_regularizer=l2_conv)(x) 

    y = BatchNormalization(gamma_regularizer=l2_batch_gamma ,beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)
#y = Dropout(rate=drop_rate)(y)

    y = MaxPooling1D(pool_size = n_pool, padding=padding_pool)(y) ##
    shortcut1_2 = MaxPooling1D(pool_size = n_pool, padding=padding_pool)(y)## поправить
    shortcut1_3 = MaxPooling1D(pool_size = n_pool, padding=padding_pool)(shortcut1_2)##
    shortcut1_4 = MaxPooling1D(pool_size = n_pool, padding=padding_pool)(shortcut1_3)##
    shortcut1_5 = MaxPooling1D(pool_size = n_pool, padding=padding_pool)(shortcut1_4)

    # второй spider-block
    y = Conv1D(filters=n_filters*2, kernel_size=n_kernel, strides=n_strides, padding='same',
              use_bias=_bias, kernel_regularizer=l2_conv)(y) 
    y = BatchNormalization(gamma_regularizer=l2_batch_gamma ,beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)

    y = MaxPooling1D(pool_size = n_pool, padding=padding_pool)(y) ##
    shortcut2_2 = MaxPooling1D(pool_size = n_pool, padding=padding_pool)(y)##
    shortcut2_3 = MaxPooling1D(pool_size = n_pool, padding=padding_pool)(shortcut2_2)##
    shortcut2_4 = MaxPooling1D(pool_size = n_pool, padding=padding_pool)(shortcut2_3)


    y= Concatenate(axis=-1)([shortcut1_2, y])
    n = n+1

    # третий spider-block
    y = Conv1D(filters=n_filters*(3 + 1), kernel_size=n_kernel, strides=n_strides, padding='same',
              use_bias=_bias, kernel_regularizer=l2_conv)(y) 
    y = BatchNormalization(gamma_regularizer=l2_batch_gamma ,beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)

    y = MaxPooling1D(pool_size = n_pool, padding=padding_pool)(y) ##
    shortcut3_2 = MaxPooling1D(pool_size = n_pool, padding=padding_pool)(y)##
    shortcut3_3 = MaxPooling1D(pool_size = n_pool, padding=padding_pool)(shortcut3_2)##

    y= Concatenate(axis=-1)([shortcut2_2, shortcut1_3, y])
    n = n+1
    # четвертый spider-block
    y = Conv1D(filters=n_filters*(4 + 1), kernel_size=n_kernel, strides=n_strides, padding='same',
              use_bias=_bias, kernel_regularizer=l2_conv)(y) 
    y = BatchNormalization(gamma_regularizer=l2_batch_gamma ,beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)

    y = MaxPooling1D(pool_size = n_pool, padding=padding_pool)(y) ##
    shortcut4_2 = MaxPooling1D(pool_size = n_pool, padding=padding_pool)(y)##

    y= Concatenate(axis=-1)([shortcut3_2, shortcut2_3, shortcut1_4, y])
    n = n+1
    y = Dropout(drop_out_conv*(n**drop_out_rate))(y)
    print(n, drop_out_conv*(n**drop_out_rate))

    # пятый spider-block
    y = Conv1D(filters=n_filters*(5 + 1), kernel_size=n_kernel, strides=n_strides, padding='same',
              use_bias=_bias, kernel_regularizer=l2_conv)(y) 
    y = BatchNormalization(gamma_regularizer=l2_batch_gamma ,beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)

    y = MaxPooling1D(pool_size = n_pool, padding=padding_pool)(y) ##
    y= Concatenate(axis=-1)([shortcut4_2, shortcut3_3, shortcut2_4, shortcut1_5, y])
    n = n+1
    y = Dropout(drop_out_conv*(n**drop_out_rate))(y)
    print(n, drop_out_conv*(n**drop_out_rate))

    # шестой spider-block
    y = Conv1D(filters=n_filters*(6 + 1), kernel_size=n_kernel, strides=n_strides, padding='same',
              use_bias=_bias, kernel_regularizer=l2_conv)(y) 
    y = BatchNormalization(gamma_regularizer=l2_batch_gamma ,beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)
    if gl_pool_max:
        z = GlobalMaxPooling1D()(y)
        
    else:
        z = GlobalAveragePooling1D()(y)
    #z = Flatten()(y) #сглаживание, пример использования - https://stackoverflow.com/questions/43237124/what-is-the-role-of-flatten-in-keras
    z = Dense(hidden, activation='relu')(z)
    z = Dropout(drop_out)(z)
    z = Dense(hidden, activation='relu')(z)
    z = Dropout(drop_out)(z)
    predictions = Dense(classes, activation='softmax')(z)

    #model = Sequential()
    model_15 = Model(inputs=x, outputs=predictions)
    
    return model_15

In [57]:
gl_p = [True]
l2_batch = [1E-5]
n_ker = [5]
n_fil = [ 5]
d_hidden = [64] 
drop_out = [ 0.25]
drop_out_conv = [0.001]

In [59]:
param = itertools.product(gl_p, l2_batch, n_ker, n_fil, d_hidden, drop_out, drop_out_conv, reg_dense)

In [44]:
i = 0
for p in param:
    print(i, p)
    i += 1

0 (True, 1e-05, 5, 5, 64, 0.25, 0.001, 1e-05)


In [46]:
inp_shape

124

In [50]:
for p in param:
    print('=====================================================================================================')
    print(inp_shape, X_2_2.shape, p)
    
    model_grid = spider_model(gl_pool_max = p[0], reg = p[1],
                             n_kernel = p[2], n_filters = p[3], hidden = p[4], drop_out = p[5], drop_out_conv = p[6],
                             n_features = inp_shape)
    
    model_grid.compile(loss='categorical_crossentropy', 
                 optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
              metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
    model_grid.summary()
    

124 (185400, 124) (True, 1e-05, 5, 5, 64, 0.25, 0.001, 1e-05)
3 0.081
4 0.256
Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 124, 1)]     0                                            
__________________________________________________________________________________________________
conv1d_12 (Conv1D)              (None, 124, 5)       30          input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_12 (BatchNo (None, 124, 5)       20          conv1d_12[0][0]                  
__________________________________________________________________________________________________
re_lu_12 (ReLU)                 (None, 124, 5)       0           batch_normalization_12[0][0]     
_________

In [60]:
training_generator = DataGenerator(train_for, x_col, y_col, batch_size=512, alpha = None, class_w = 0.05 )
print(len(x_col), y_col )

124 BAD_FLAG


In [62]:
def lr_step_decay(epoch, lr):
    drop_rate = 0.1
    epochs_drop = 20.0
    initial_learning_rate = 0.005
    
    return initial_learning_rate * math.pow(drop_rate, math.floor(epoch/epochs_drop))

In [63]:
VERBOSE = 2
BATCH_SIZE = 512
NB_EPOCH = 150
class_weighting = {0:0.05 , 1:1}

result_all_8 = pd.DataFrame()
result_all_8['name_model'] = None
result_all_8['params'] = None
result_all_8['val_GINI'] = None
result_all_8['val_APS'] = None
result_all_8['test_GINI'] = None
result_all_8['test_APS'] = None
j = 0

for p in param:
    print('=====================================================================================================')
    print(inp_shape, X_2_2.shape, p)
    
    model_grid = spider_model(gl_pool_max = p[0], reg = p[1],
                             n_kernel = p[2], n_filters = p[3], hidden = p[4], drop_out = p[5], drop_out_conv = p[6],
                             n_features = inp_shape)

    
    model_grid.compile(loss='categorical_crossentropy', 
                 optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
              metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
    
    history_XX = model_grid.fit_generator(generator=training_generator,  #training_aug,
                    validation_data= (X_2_2, Y_test_2) , #Y_test_2 = np_utils.to_categorical( y_val, 2) 
                                     epochs=NB_EPOCH, verbose=VERBOSE, class_weight=class_weighting,
                                     callbacks=[_time , EarlyStopping(monitor='val_loss', patience=20),
                                               Metrics(valid_data=(X_2_2, Y_test_2))
                                               , LearningRateScheduler(lr_step_decay, verbose=1)])
    
    res_model_ = pd.DataFrame(history_XX.history, columns = history_XX.history.keys())
    dd = str(200000 + datetime.now().month*100 + datetime.now().day) + '_' + str(datetime.now().hour)
    name_m = 'model_spider6_internal_wo_btest_'  + str(p[2]) + '_' + str(p[3]) + '_' + str(p[4]) + '_' + str(p[1]) + '_' + str(dd)
    
    model_grid.save( name_m + '.h5')
    res_model_.to_csv(name_m + '.csv')
    
    predict_class_val = model_grid.predict(X_2_2)
    APS = metrics.average_precision_score(y_val, predict_class_val[:,1])
    GINI = 2*(metrics.roc_auc_score(y_val , predict_class_val[:,1])) - 1

    predict_class_test = model_grid.predict(X_3_2)
    APS_t = metrics.average_precision_score(y_test, predict_class_test[:,1])
    GINI_t = 2*(metrics.roc_auc_score(y_test, predict_class_test[:,1])) - 1
    
    result_all_8.at[j , 'name_model'] = name_m
    result_all_8.at[j ,'params'] = str(p)
    result_all_8.at[j ,'val_GINI'] = GINI
    result_all_8.at[j ,'val_APS'] = APS
    result_all_8.at[j ,'test_GINI'] = GINI_t
    result_all_8.at[j ,'test_APS'] = APS_t
    
    result_all_8.to_csv('internal_spider_wo_btest_6.csv')
    j += 1
    print(name_m, 'valid_for_train: ', GINI, APS, '| test: ', GINI_t, APS_t) 

124 (185400, 124) (True, 1e-05, 5, 5, 64, 0.25, 0.001, 1e-05)
3 0.081
4 0.256
train_begin | time:  12.05.2021 00:27:54

Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/150
epoch_end | time:  12.05.2021 00:40:24
 — val_aps:  0.071814 — val_a: 0.933132
2945/2945 - 787s - loss: 0.0109 - accuracy: 0.9677 - auc_3: 0.9924 - precision_3: 0.9677 - recall_3: 0.9677 - val_loss: 0.0275 - val_accuracy: 0.9951 - val_auc_3: 0.9995 - val_precision_3: 0.9951 - val_recall_3: 0.9951

Epoch 00002: LearningRateScheduler reducing learning rate to 0.005.
Epoch 2/150
epoch_end | time:  12.05.2021 00:52:51
 — val_aps:  0.074780 — val_a: 0.945938
2945/2945 - 746s - loss: 0.0069 - accuracy: 0.9852 - auc_3: 0.9971 - precision_3: 0.9852 - recall_3: 0.9852 - val_loss: 0.0380 - val_accuracy: 0.9908 - val_auc_3: 0.9989 - val_precision_3: 0.9908 - val_recall_3: 0.9908

Epoch 00003: LearningRateScheduler reducing learning rate to 0.005.
Epoch 3/150
epoch_end | time:  12.05.2021 01:05:21
 — 


Epoch 00022: LearningRateScheduler reducing learning rate to 0.0005.
Epoch 22/150
epoch_end | time:  12.05.2021 04:57:15
 — val_aps:  0.075493 — val_a: 0.948222
2945/2945 - 656s - loss: 0.0036 - accuracy: 0.9898 - auc_3: 0.9994 - precision_3: 0.9898 - recall_3: 0.9898 - val_loss: 0.0204 - val_accuracy: 0.9954 - val_auc_3: 0.9995 - val_precision_3: 0.9954 - val_recall_3: 0.9954

Epoch 00023: LearningRateScheduler reducing learning rate to 0.0005.
Epoch 23/150
epoch_end | time:  12.05.2021 05:09:06
 — val_aps:  0.076447 — val_a: 0.947662
2945/2945 - 725s - loss: 0.0036 - accuracy: 0.9895 - auc_3: 0.9994 - precision_3: 0.9895 - recall_3: 0.9895 - val_loss: 0.0196 - val_accuracy: 0.9961 - val_auc_3: 0.9995 - val_precision_3: 0.9961 - val_recall_3: 0.9961

Epoch 00024: LearningRateScheduler reducing learning rate to 0.0005.
Epoch 24/150
epoch_end | time:  12.05.2021 05:21:12
 — val_aps:  0.074972 — val_a: 0.945920
2945/2945 - 710s - loss: 0.0035 - accuracy: 0.9894 - auc_3: 0.9994 - precisi

In [67]:
gl_p = [True]
l2_batch = [0.0001]
n_ker = [3]
n_fil = [10]
d_hidden = [100] 
drop_out = [ 0.25]
drop_out_conv = [0.001]

In [68]:
param_2 = itertools.product(gl_p, l2_batch, n_ker, n_fil, d_hidden, drop_out, drop_out_conv, reg_dense)

In [66]:
VERBOSE = 2
BATCH_SIZE = 512
NB_EPOCH = 150
class_weighting = {0:0.05 , 1:1}

result_all_8 = pd.DataFrame()
result_all_8['name_model'] = None
result_all_8['params'] = None
result_all_8['val_GINI'] = None
result_all_8['val_APS'] = None
result_all_8['test_GINI'] = None
result_all_8['test_APS'] = None
j = 0

for p in param_2:
    print('=====================================================================================================')
    print(inp_shape, X_2_2.shape, p)
    
    model_grid = spider_model(gl_pool_max = p[0], reg = p[1],
                             n_kernel = p[2], n_filters = p[3], hidden = p[4], drop_out = p[5], drop_out_conv = p[6],
                             n_features = inp_shape)

    
    model_grid.compile(loss='categorical_crossentropy', 
                 optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
              metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
    
    history_XX = model_grid.fit_generator(generator=training_generator,  #training_aug,
                    validation_data= (X_2_2, Y_test_2) , #Y_test_2 = np_utils.to_categorical( y_val, 2) 
                                     epochs=NB_EPOCH, verbose=VERBOSE, class_weight=class_weighting,
                                     callbacks=[_time , EarlyStopping(monitor='val_loss', patience=20),
                                               Metrics(valid_data=(X_2_2, Y_test_2))
                                               , LearningRateScheduler(lr_step_decay, verbose=1)])
    
    res_model_ = pd.DataFrame(history_XX.history, columns = history_XX.history.keys())
    dd = str(200000 + datetime.now().month*100 + datetime.now().day) + '_' + str(datetime.now().hour)
    name_m = 'RIGHT_model_spider6_internal_wo_btest_'  + str(p[2]) + '_' + str(p[3]) + '_' + str(p[4]) + '_' + str(p[1]) + '_' + str(dd)
    
    model_grid.save( name_m + '.h5')
    res_model_.to_csv(name_m + '.csv')
    
    predict_class_val = model_grid.predict(X_2_2)
    APS = metrics.average_precision_score(y_val, predict_class_val[:,1])
    GINI = 2*(metrics.roc_auc_score(y_val , predict_class_val[:,1])) - 1

    predict_class_test = model_grid.predict(X_3_2)
    APS_t = metrics.average_precision_score(y_test, predict_class_test[:,1])
    GINI_t = 2*(metrics.roc_auc_score(y_test, predict_class_test[:,1])) - 1
    
    result_all_8.at[j , 'name_model'] = name_m
    result_all_8.at[j ,'params'] = str(p)
    result_all_8.at[j ,'val_GINI'] = GINI
    result_all_8.at[j ,'val_APS'] = APS
    result_all_8.at[j ,'test_GINI'] = GINI_t
    result_all_8.at[j ,'test_APS'] = APS_t
    
    result_all_8.to_csv('RIGHT_internal_spider_wo_btest_6.csv')
    j += 1
    print(name_m, 'valid_for_train: ', GINI, APS, '| test: ', GINI_t, APS_t) 

124 (185400, 124) (True, 0.0001, 3, 10, 100, 0.25, 0.001, 1e-05)
3 0.081
4 0.256
train_begin | time:  12.05.2021 10:50:17

Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/150
epoch_end | time:  12.05.2021 11:08:08
 — val_aps:  0.056843 — val_a: 0.934411
2945/2945 - 1092s - loss: 0.0135 - accuracy: 0.9657 - auc_4: 0.9921 - precision_4: 0.9657 - recall_4: 0.9657 - val_loss: 0.0633 - val_accuracy: 0.9842 - val_auc_4: 0.9971 - val_precision_4: 0.9842 - val_recall_4: 0.9842

Epoch 00002: LearningRateScheduler reducing learning rate to 0.005.
Epoch 2/150
epoch_end | time:  12.05.2021 11:27:04
 — val_aps:  0.056783 — val_a: 0.941033
2945/2945 - 1162s - loss: 0.0073 - accuracy: 0.9836 - auc_4: 0.9967 - precision_4: 0.9836 - recall_4: 0.9836 - val_loss: 0.0609 - val_accuracy: 0.9846 - val_auc_4: 0.9974 - val_precision_4: 0.9846 - val_recall_4: 0.9846

Epoch 00003: LearningRateScheduler reducing learning rate to 0.005.
Epoch 3/150
epoch_end | time:  12.05.2021 11:48:5


Epoch 00022: LearningRateScheduler reducing learning rate to 0.0005.
Epoch 22/150
epoch_end | time:  12.05.2021 18:51:29
 — val_aps:  0.090559 — val_a: 0.949654
2945/2945 - 1732s - loss: 0.0041 - accuracy: 0.9897 - auc_4: 0.9993 - precision_4: 0.9897 - recall_4: 0.9897 - val_loss: 0.0214 - val_accuracy: 0.9957 - val_auc_4: 0.9995 - val_precision_4: 0.9957 - val_recall_4: 0.9957

Epoch 00023: LearningRateScheduler reducing learning rate to 0.0005.
Epoch 23/150
epoch_end | time:  12.05.2021 19:19:30
 — val_aps:  0.091404 — val_a: 0.948689
2945/2945 - 1687s - loss: 0.0039 - accuracy: 0.9899 - auc_4: 0.9993 - precision_4: 0.9899 - recall_4: 0.9899 - val_loss: 0.0227 - val_accuracy: 0.9949 - val_auc_4: 0.9995 - val_precision_4: 0.9949 - val_recall_4: 0.9949

Epoch 00024: LearningRateScheduler reducing learning rate to 0.0005.
Epoch 24/150
epoch_end | time:  12.05.2021 19:45:48
 — val_aps:  0.085042 — val_a: 0.946010
2945/2945 - 1580s - loss: 0.0038 - accuracy: 0.9895 - auc_4: 0.9993 - prec

In [69]:
#model_spider6_gridTrue_None_7_10_30.h5 - лучшая модель по Profit-метрике!!!
gl_p = [True]
l2_batch = [None]
n_ker = [7]
n_fil = [10]
d_hidden = [30] 
drop_out = [ 0.25]
drop_out_conv = [0.001]

In [70]:
param_2 = itertools.product(gl_p, l2_batch, n_ker, n_fil, d_hidden, drop_out, drop_out_conv, reg_dense)

In [71]:
VERBOSE = 2
BATCH_SIZE = 512
NB_EPOCH = 150
class_weighting = {0:0.05 , 1:1}

result_all_8 = pd.DataFrame()
result_all_8['name_model'] = None
result_all_8['params'] = None
result_all_8['val_GINI'] = None
result_all_8['val_APS'] = None
result_all_8['test_GINI'] = None
result_all_8['test_APS'] = None
j = 0

for p in param_2:
    print('=====================================================================================================')
    print(inp_shape, X_2_2.shape, p)
    
    model_grid = spider_model(gl_pool_max = p[0], reg = p[1],
                             n_kernel = p[2], n_filters = p[3], hidden = p[4], drop_out = p[5], drop_out_conv = p[6],
                             n_features = inp_shape)

    
    model_grid.compile(loss='categorical_crossentropy', 
                 optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
              metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
    
    history_XX = model_grid.fit_generator(generator=training_generator,  #training_aug,
                    validation_data= (X_2_2, Y_test_2) , #Y_test_2 = np_utils.to_categorical( y_val, 2) 
                                     epochs=NB_EPOCH, verbose=VERBOSE, class_weight=class_weighting,
                                     callbacks=[_time , EarlyStopping(monitor='val_loss', patience=20),
                                               Metrics(valid_data=(X_2_2, Y_test_2))
                                               , LearningRateScheduler(lr_step_decay, verbose=1)])
    
    res_model_ = pd.DataFrame(history_XX.history, columns = history_XX.history.keys())
    dd = str(200000 + datetime.now().month*100 + datetime.now().day) + '_' + str(datetime.now().hour)
    name_m = 'RIGHT_model_spider6_internal_wo_btest_'  + str(p[2]) + '_' + str(p[3]) + '_' + str(p[4]) + '_' + str(p[1]) + '_' + str(dd)
    
    model_grid.save( name_m + '.h5')
    res_model_.to_csv(name_m + '.csv')
    
    predict_class_val = model_grid.predict(X_2_2)
    APS = metrics.average_precision_score(y_val, predict_class_val[:,1])
    GINI = 2*(metrics.roc_auc_score(y_val , predict_class_val[:,1])) - 1

    predict_class_test = model_grid.predict(X_3_2)
    APS_t = metrics.average_precision_score(y_test, predict_class_test[:,1])
    GINI_t = 2*(metrics.roc_auc_score(y_test, predict_class_test[:,1])) - 1
    
    result_all_8.at[j , 'name_model'] = name_m
    result_all_8.at[j ,'params'] = str(p)
    result_all_8.at[j ,'val_GINI'] = GINI
    result_all_8.at[j ,'val_APS'] = APS
    result_all_8.at[j ,'test_GINI'] = GINI_t
    result_all_8.at[j ,'test_APS'] = APS_t
    
    result_all_8.to_csv('RIGHT_internal_spider_wo_btest_6.csv')
    j += 1
    print(name_m, 'valid_for_train: ', GINI, APS, '| test: ', GINI_t, APS_t) 

124 (185400, 124) (True, None, 7, 10, 30, 0.25, 0.001, 1e-05)
3 0.081
4 0.256
train_begin | time:  13.05.2021 10:56:09

Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/150
epoch_end | time:  13.05.2021 11:26:05
 — val_aps:  0.058550 — val_a: 0.946347
2945/2945 - 1857s - loss: 0.0101 - accuracy: 0.9669 - auc_5: 0.9926 - precision_5: 0.9669 - recall_5: 0.9669 - val_loss: 0.0475 - val_accuracy: 0.9932 - val_auc_5: 0.9991 - val_precision_5: 0.9932 - val_recall_5: 0.9932

Epoch 00002: LearningRateScheduler reducing learning rate to 0.005.
Epoch 2/150
epoch_end | time:  13.05.2021 11:51:44
 — val_aps:  0.065242 — val_a: 0.946175
2945/2945 - 1515s - loss: 0.0058 - accuracy: 0.9883 - auc_5: 0.9978 - precision_5: 0.9883 - recall_5: 0.9883 - val_loss: 0.0479 - val_accuracy: 0.9927 - val_auc_5: 0.9992 - val_precision_5: 0.9927 - val_recall_5: 0.9927

Epoch 00003: LearningRateScheduler reducing learning rate to 0.005.
Epoch 3/150
epoch_end | time:  13.05.2021 12:15:37
 


Epoch 00022: LearningRateScheduler reducing learning rate to 0.0005.
Epoch 22/150
epoch_end | time:  13.05.2021 20:57:54
 — val_aps:  0.069421 — val_a: 0.928581
2945/2945 - 1738s - loss: 0.0019 - accuracy: 0.9916 - auc_5: 0.9997 - precision_5: 0.9916 - recall_5: 0.9916 - val_loss: 0.0268 - val_accuracy: 0.9956 - val_auc_5: 0.9985 - val_precision_5: 0.9956 - val_recall_5: 0.9956

Epoch 00023: LearningRateScheduler reducing learning rate to 0.0005.
Epoch 23/150
epoch_end | time:  13.05.2021 21:26:45
 — val_aps:  0.068958 — val_a: 0.922025
2945/2945 - 1725s - loss: 0.0018 - accuracy: 0.9917 - auc_5: 0.9997 - precision_5: 0.9917 - recall_5: 0.9917 - val_loss: 0.0292 - val_accuracy: 0.9962 - val_auc_5: 0.9983 - val_precision_5: 0.9962 - val_recall_5: 0.9962

Epoch 00024: LearningRateScheduler reducing learning rate to 0.0005.
Epoch 24/150
epoch_end | time:  13.05.2021 21:55:45
 — val_aps:  0.071576 — val_a: 0.922451
2945/2945 - 1742s - loss: 0.0017 - accuracy: 0.9921 - auc_5: 0.9997 - prec